In [1]:
import pandas as pd 
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.api as sms
plt.style.use('ggplot')
data = pd.read_csv('bg_master_1.csv')
# data.head(10)
# data.columns


In [9]:
# data.hist(figsize=(18,10) );
data['ENTRIES'][321] = (data.iloc[320].ENTRIES + data.iloc[322].ENTRIES) // 2
data['ENTRIES'][173] = (data.iloc[172].ENTRIES + data.iloc[174].ENTRIES) // 2
data['ENTRIES'][180] = (data.iloc[179].ENTRIES + data.iloc[181].ENTRIES) // 2
data['ENTRIES'][321] = (data.iloc[320].ENTRIES + data.iloc[322].ENTRIES) // 2
data['ENTRIES'][211] = (data.iloc[204].ENTRIES + data.iloc[218].ENTRIES) // 2 # Saturday
data['ENTRIES'][212] = (data.iloc[205].ENTRIES + data.iloc[219].ENTRIES) // 2
data['ENTRIES'][323] = (data['ENTRIES'][316] + data['ENTRIES'][330])// 2#data.loc[316].ENTRIES # Saturday
data['ENTRIES'][324] = (data['ENTRIES'][317] + data['ENTRIES'][331]) // 2
# consider refactoring to use .loc to avoide warnings 
# consider generalizing into helper function "turnstile_clean()"

def is_weekend(date):
    if date.weekday() >= 5:
        return 1
    else: 
        return 0 
    
data['DATE2'] = pd.to_datetime(data.DATE2)
data['is_weekend'] = data.apply(lambda r: is_weekend(r.DATE2), axis=1)
wkend_entries = data[data.is_weekend == 1]
weekday_entries = data[data.is_weekend == 0]
features = ['PRCP','SNOW','SNWD','TMAX','TMIN']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ip

In [10]:
def feature_plots_with_graphs(df):
    results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value', 'normality (JB)' ]]
    for feature in features: 
        f = 'ENTRIES~' + feature
        print('\n', "Subway Weekend - Regression Analysis and Diagnostics for formula: ",f)
        print("--------------------------------------------------------------------------------------")
        model = ols(formula=f, data=df).fit()
        df_min_max = pd.DataFrame({feature: [df[feature].min(), df[feature].max()]})
        preds = model.predict(df_min_max)
        df.plot(kind='scatter', x=feature, y='ENTRIES')
        plt.plot(df_min_max, preds, c='red', linewidth=2)
        plt.show()
        fig = plt.figure(figsize=(15,8))
        fig = sm.graphics.plot_regress_exog(model, feature, fig=fig)
        fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True,   )
        plt.show()
        results.append([feature, model.rsquared, model.params[0], model.params[1], model.pvalues[1], sms.jarque_bera(model.resid)[0] ])    
    return results

def feature_results_summary(df):
    results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value', 'normality (JB)' ]]
    for feature in features: 
        f = 'ENTRIES~' + feature
        model = ols(formula=f, data=df).fit()
        df_min_max = pd.DataFrame({feature: [df[feature].min(), df[feature].max()]})
        preds = model.predict(df_min_max)
        results.append([feature, model.rsquared, model.params[0], model.params[1], model.pvalues[1], sms.jarque_bera(model.resid)[0] ])    
    return results

In [12]:
weekend_results = pd.DataFrame(feature_results_summary(wkend_entries))
weekend_results

,0,1,2,3,4,5
0,ind_var,r_squared,intercept,slope,p-value,normality (JB)
1,PRCP,0.0537471,8722.7,-1196.74,0.0178827,3.29887
2,SNOW,0.00511428,8489.77,235.885,0.470655,2.28656
3,SNWD,0.0114773,8563.85,-225.663,0.279052,1.80008
4,TMAX,0.111539,6261.62,36.7602,0.000530711,3.19333
5,TMIN,0.0641931,7062.96,30.2564,0.00945711,2.61552


In [13]:
weekday_results = pd.DataFrame(feature_results_summary(weekday_entries))
weekday_results

,0,1,2,3,4,5
0,ind_var,r_squared,intercept,slope,p-value,normality (JB)
1,PRCP,0.00982815,28921.1,-1213.98,0.110074,1855.46
2,SNOW,0.0633981,28894.5,-1200.91,3.87596e-05,2157.2
3,SNWD,0.00183272,28763,-204.191,0.491064,1760.08
4,TMAX,0.0367809,25845.9,45.9597,0.00185555,1476.89
5,TMIN,0.0289766,26599.5,43.0837,0.00583306,1492.22


In [37]:
# features = ['PRCP','SNOW','SNWD','TMAX','TMIN']
# for column in features:
#     plt.scatter(data[column], data.ENTRIES, label=column)
#     plt.legend()
#     plt.show()



In [4]:
# data.ENTRIES[data.ENTRIES > 45000]
# data.iloc[172], data.iloc[173], data.iloc[174]

# data.iloc[173].ENTRIES.replace(data.iloc[173].ENTRIES, (data.iloc[172].ENTRIES + data.iloc[174].ENTRIES) // 2)
data['ENTRIES'][173] = (data.iloc[172].ENTRIES + data.iloc[174].ENTRIES) // 2

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
data['ENTRIES'][180] = (data.iloc[179].ENTRIES + data.iloc[181].ENTRIES) // 2

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
data['ENTRIES'][321] = (data.iloc[320].ENTRIES + data.iloc[322].ENTRIES) // 2

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
data['ENTRIES'][211]

45

In [8]:
# Weekend
data['ENTRIES'][211] = (data.iloc[204].ENTRIES + data.iloc[218].ENTRIES) // 2 # Saturday
data['ENTRIES'][212] = (data.iloc[205].ENTRIES + data.iloc[219].ENTRIES) // 2

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
data.loc[218]

Unnamed: 0           218
DATE2         2018-07-07
ENTRIES            10031
EXITS              11174
PRCP                   0
SNOW                   0
SNWD                   0
TMAX                  78
TMIN                  62
Name: 218, dtype: object

In [10]:
data['ENTRIES'][323] = (data['ENTRIES'][316] + data['ENTRIES'][330])// 2#data.loc[316].ENTRIES # Saturday
data.loc[323] # Sunday

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Unnamed: 0           323
DATE2         2018-10-20
ENTRIES             6830
EXITS                 34
PRCP                0.06
SNOW                   0
SNWD                   0
TMAX                  62
TMIN                  51
Name: 323, dtype: object

In [11]:
data['ENTRIES'][324] = (data['ENTRIES'][317] + data['ENTRIES'][331]) // 2
data['ENTRIES'][324]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


7168

In [50]:
# data[data.is_weekend == 0]

In [12]:
def is_weekend(date):
    if date.weekday() >= 5:
        return 1
    else: 
        return 0 

Timestamp('2017-12-05 00:00:00')

In [13]:
data['DATE2'] = pd.to_datetime(data.DATE2)

In [25]:
wkend_entries = data[data.is_weekend == 1]
weekday_entries = data[data.is_weekend == 0]
# wkend_entries.ENTRIES

In [14]:
features = ['PRCP','SNOW','SNWD','TMAX','TMIN']
# for column in features:
#     plt.scatter(weekday_entries[column], weekday_entries.ENTRIES, label=column)
#     plt.legend()
#     plt.show()

In [15]:
# data['is_weekend'] = data.apply(lambda r: is_weekend(r.DATE2), axis=1)

In [31]:
# import scipy.stats as stats
# import statsmodels.stats.api as sms

# def feature_plots_with_graphs(df):
#     results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value', 'normality (JB)' ]]
#     for feature in features: 
#         f = 'ENTRIES~' + feature
#         print('\n', "Subway Weekend - Regression Analysis and Diagnostics for formula: ",f)
#         print("--------------------------------------------------------------------------------------")
#         model = ols(formula=f, data=df).fit()
#         df_min_max = pd.DataFrame({feature: [df[feature].min(), df[feature].max()]})
#         preds = model.predict(df_min_max)
#         df.plot(kind='scatter', x=feature, y='ENTRIES')
#         plt.plot(df_min_max, preds, c='red', linewidth=2)
#         plt.show()
#         fig = plt.figure(figsize=(15,8))
#         fig = sm.graphics.plot_regress_exog(model, feature, fig=fig)
#         fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True,   )
#         plt.show()
#         results.append([feature, model.rsquared, model.params[0], model.params[1], model.pvalues[1], sms.jarque_bera(model.resid)[0] ])    
#     return results

# def feature_plots_no_graphs(df):
#     results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value', 'normality (JB)' ]]
#     for feature in features: 
#         f = 'ENTRIES~' + feature
# #         print('\n', "Regression Analysis and Diagnostics for formula: ",f)
# #         print("--------------------------------------------------------------------------------------")
#         model = ols(formula=f, data=df).fit()
#         df_min_max = pd.DataFrame({feature: [df[feature].min(), df[feature].max()]})
#         preds = model.predict(df_min_max)
# #         df.plot(kind='scatter', x=feature, y='ENTRIES')
# #          plt.plot(df_min_max, preds, c='red', linewidth=2)
# #         plt.show()
# #         fig = plt.figure(figsize=(15,8))
# #         fig = sm.graphics.plot_regress_exog(model, feature, fig=fig)
# #         fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True,   )
# #         plt.show()
#         results.append([feature, model.rsquared, model.params[0], model.params[1], model.pvalues[1], sms.jarque_bera(model.resid)[0] ])    
#     return results


In [24]:
# weekend_results = pd.DataFrame(results)
# weekend_results.head()

,0,1,2,3,4,5
0,ind_var,r_squared,intercept,slope,p-value,normality (JB)
1,PRCP,0.0537471,8722.7,-1196.74,0.0178827,3.29887
2,SNOW,0.00511428,8489.77,235.885,0.470655,2.28656
3,SNWD,0.0114773,8563.85,-225.663,0.279052,1.80008
4,TMAX,0.111539,6261.62,36.7602,0.000530711,3.19333


In [33]:
# weekday_results = pd.DataFrame(feature_plots_no_graphs(weekday_entries))

In [34]:
# weekday_results

,0,1,2,3,4,5
0,ind_var,r_squared,intercept,slope,p-value,normality (JB)
1,PRCP,0.00982815,28921.1,-1213.98,0.110074,1855.46
2,SNOW,0.0633981,28894.5,-1200.91,3.87596e-05,2157.2
3,SNWD,0.00183272,28763,-204.191,0.491064,1760.08
4,TMAX,0.0367809,25845.9,45.9597,0.00185555,1476.89
5,TMIN,0.0289766,26599.5,43.0837,0.00583306,1492.22


In [35]:
# weekend_results 

,0,1,2,3,4,5
0,ind_var,r_squared,intercept,slope,p-value,normality (JB)
1,PRCP,0.0537471,8722.7,-1196.74,0.0178827,3.29887
2,SNOW,0.00511428,8489.77,235.885,0.470655,2.28656
3,SNWD,0.0114773,8563.85,-225.663,0.279052,1.80008
4,TMAX,0.111539,6261.62,36.7602,0.000530711,3.19333
5,TMIN,0.0641931,7062.96,30.2564,0.00945711,2.61552
